In [1]:
from scipy.stats import norm

In [2]:
import yfinance as yf, math, pandas as pd, numpy as np

In [3]:
from datetime import datetime

In [4]:
def cdf(x):
    return 0.5*(1+math.erf(x/math.sqrt(2)))
            

In [5]:
def d1(S,K, r, t, vol):
    return (math.log(S/K)+(r + (vol**2)/2)*t)/(vol*math.sqrt(t))

In [6]:
def d2(S, K, r, t, vol):
    return d1(S, K, r, t, vol) - vol*math.sqrt(t)

In [7]:
def call(S, K, r, t, vol):
    Nd1 = cdf(d1(S, K, r, t, vol))
    Nd2= cdf(d2(S, K, r, t, vol))
    return S*Nd1-K*math.exp(-r*t)*Nd2

In [8]:
def put(S, K, r, t, vol):
    Nd1 = cdf(-d1(S, K, r, t, vol))
    Nd2 = cdf(-d2(S, K, r, t, vol))
    return K*math.exp(-r*t)*Nd2 - S*Nd1

In [9]:
def optiondata(symbol, date):
    option=yf.Ticker(symbol)
    options_data=option.option_chain(date)
    return options_data.calls, options_data.puts

In [10]:
def getS(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    return stock.history(period="1d")['Close'].iloc[-1]

In [11]:
def getrate():
    symbol = "^TNX"
    now=datetime.now()
    ago10 = now.replace(year=now.year-10)
    treasury_data = yf.download(symbol, start=ago10, end=now)
    last_yield = treasury_data['Close'].iloc[-1]
    return last_yield

In [12]:
def get_t(date_str):
    date_obj=datetime.strptime(date_str, '%Y-%m-%d')
    diff=(date_obj-datetime.now()).days/365.25
    return diff

In [13]:
def calculate_vol(symb):
    data = yf.download(symb, start=datetime.now().replace(year=datetime.now().year-10), end=datetime.now())
    data['Daily_Return'] = data['Adj Close'].pct_change()
    daily_vol = data['Daily_Return'].std()
    annual_vol=daily_vol*252**0.5
    return annual_vol
    

In [14]:
def delta_call(S, k, r, t, vol):
    return norm.cdf(d1(S, k, r, t, vol))

In [15]:
def delta_put(S, k, r, t, vol):
    return norm.cdf(d1(S, k, r, t, vol))-1

In [16]:
def gamma_call(S, k, r, t, vol):
    return norm.pdf(d1(S, k, r, t, vol))/(S*vol*np.sqrt(t))

In [17]:
def gamma_put(S, k, r, t, vol):
    return norm.pdf(d1(S, k, r, t, vol))/(S*vol*np.sqrt(t))

In [18]:
def vega(S, k, r, t, vol):
    return S*np.sqrt(t)*norm.pdf(d1(S, k, r, t, vol))

In [19]:
def rho_call(S, k, r, t, vol):
    return k*t*S*norm.cdf(d2(S, k, r, t, vol))
def rho_put(S, k, r, t, vol):
    return -k*t*S*norm.cdf(-d2(S, k, r, t, vol))

In [20]:
def theta_call(S, k, r, t, vol):
    return -(S*norm.pdf(d1(S, k, r, t, vol))*vol/(2*np.sqrt(t)))-r*k*np.exp(-r*t)*norm.cdf(d2(S, k, r, t, vol))
def theta_put(S, k, r, t, vol):
    return -(S*norm.pdf(d1(S, k, r, t, vol))*vol/(2*np.sqrt(t)))+r*k*np.exp(-r*t)*norm.cdf(-d2(S, k, r, t, vol))

In [21]:
symbol ='AAPL'
S= getS(symbol)
vol=calculate_vol(symbol)
r=getrate()/100
vol=calculate_vol(symbol)
option = yf.Ticker(symbol)
print(f"Risk free rate {r}\nCurrent Stock Price {S}\nVolatility {vol}")
combined = pd.DataFrame()
for i in range(20):
    date1=option.options[i]
    t=get_t(date1)
    calls,put=optiondata(symbol, date1)
    col_drop=['lastTradeDate', 'lastPrice', 'volume', 'openInterest', 'contractSize', 'currency']
    calls.drop(columns=col_drop, inplace=True)
    calls['bsmValuation'] = calls.apply(lambda row: call(S, row['strike'], t, r, vol), axis=1)
    calls['diff']=calls.apply(lambda row: (row['bsmValuation']-row['ask']), axis=1)
    calls['delta']=calls.apply(lambda row: delta_call(S, row['strike'], t, r, vol), axis=1)
    calls['gamma']=calls.apply(lambda row: gamma_call(S, row['strike'], t, r, vol), axis=1)
    calls['vega']=calls.apply(lambda row: vega(S, row['strike'], t, r, vol), axis=1)
    calls['rho']=calls.apply(lambda row: rho_call(S, row['strike'], t, r, vol), axis =1)
    calls['theta']=calls.apply(lambda row: theta_call(S, row['strike'], t,r,vol), axis=1)
    combined=pd.concat([combined, calls], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Risk free rate 0.04513999938964844
Current Stock Price 192.25
Volatility 0.2831879674298865


In [22]:
combined

,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,bsmValuation,diff,delta,gamma,vega,rho,theta
0,AAPL240607C00100000,100.0,0.0,0.0,0.0,0.0,0.000010,True,9.228707e+01,9.228707e+01,1.000000e+00,5.484343e-28,2.591156e-25,8.678165e+02,-8.210508e-01
1,AAPL240607C00110000,110.0,0.0,0.0,0.0,0.0,0.000010,True,8.229078e+01,8.229078e+01,1.000000e+00,4.931332e-21,2.329879e-18,9.545981e+02,-9.031558e-01
2,AAPL240607C00120000,120.0,0.0,0.0,0.0,0.0,0.000010,True,7.229448e+01,7.229448e+01,1.000000e+00,1.229465e-15,5.808786e-13,1.041380e+03,-9.852609e-01
3,AAPL240607C00125000,125.0,0.0,0.0,0.0,0.0,0.000010,True,6.729634e+01,6.729634e+01,1.000000e+00,2.035229e-13,9.615732e-11,1.084771e+03,-1.026313e+00
4,AAPL240607C00130000,130.0,0.0,0.0,0.0,0.0,0.000010,True,6.229819e+01,6.229819e+01,1.000000e+00,1.787141e-11,8.443606e-09,1.128161e+03,-1.067366e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,AAPL261218C00310000,310.0,0.0,0.0,0.0,0.0,0.062509,False,1.718015e-09,1.718015e-09,9.468697e-10,5.046622e-10,2.384349e-07,1.754773e-06,-1.205560e-06
931,AAPL261218C00320000,320.0,0.0,0.0,0.0,0.0,0.062509,False,5.369675e-11,5.369675e-11,3.194355e-11,1.845009e-11,8.717012e-09,5.924040e-08,-4.279316e-08
932,AAPL261218C00330000,330.0,0.0,0.0,0.0,0.0,0.062509,False,1.440454e-12,1.440454e-12,9.220669e-13,5.725539e-13,2.705113e-10,1.711013e-09,-1.294764e-09
933,AAPL261218C00340000,340.0,0.0,0.0,0.0,0.0,0.062509,False,4.056684e-14,4.056684e-14,2.314787e-14,1.534966e-14,7.252168e-12,4.297543e-11,-3.395639e-11


In [24]:
combined.to_csv('data.csv', index=False)